In [1]:
import MisrToolkit as Mtk
from MisrToolkit import *
import os
import pandas as pd
import numpy as np
from matplotlib import image as pltimg, pyplot as plt  #  pyplot uses the actual RGB values as they are, more accurate than PIL


In [2]:
def find_unique_pob(atmmodel_file_fp, list_of_columns):
    
    print('input file found: %s' %(os.path.isfile(atmmodel_file_fp)))
    in_csv_df = pd.read_csv(atmmodel_file_fp)
    print(in_csv_df.columns)
    pob_unique = in_csv_df.drop_duplicates(subset=list_of_columns, keep='first', ignore_index=False)
    pob_unique.info()
    
    return pob_unique

In [3]:
# from matplotlib import image as pltimg, pyplot as plt  #  pyplot uses the actual RGB values as they are, more accurate than PIL

def plot_here_and_write(in_arr, img_dir):
    write_mode = 0
    img_label = 'path_'+path_num+'_'+'blocks_'+str(start_block)+'_'+str(end_block)+'_cam_'+cam
    img_format = ".jpg"
    %matplotlib inline 

    plt.gray() # This will show the images in grayscale as default

    plt.figure(figsize=(30,30))  # set the figure size

    plt.imshow(in_arr)
    plt.show()
    
    if (write_mode):
        print('save mode: on, we write images to disc')
        
        out_img = img_label+img_format
        out_img_fullpath = os.path.join(img_dir, out_img)
        print("-> output img is: %s" %out_img_fullpath)
        pltimg.imsave(out_img_fullpath, in_arr)
        #     plt.savefig(out_img)
    
    return 0

## set up hdf file directory

In [9]:
# hdf_dir = "/Volumes/SEAGATE/april_2016/hdf_from_PH/move_to_Mac"
# hdf_dir = "/Volumes/SEAGATE/april_2016/hdf_from_PH/move_to_Mac"
# hdf_dir = "/Volumes/Ehsan-7757225325/2016/july_2016/ellipsoid_files/elliposid_files_used_for_mapping"

hdf_dir = "/Volumes/Ehsan-7757225325/2016/april_2016/hdf_files_April2016_from_PH"
# hdf_dir = "/Volumes/Ehsan-7757225325/2016/july_2016/ellipsoid_files/elliposid_files_used_for_mapping"
# hdf_file = "MISR_AM1_GRP_ELLIPSOID_GM_P"+path_num+"_O0"+str(orbit_num)+"_"+cam+"_F03_0024.hdf"

In [10]:
path_num = '157'
orbit_num = 88131
start_block = 10
end_block = 10
corners_for_block = 23
cam = "AN"


In [11]:
hdf_file_label = "MISR_AM1_GRP_ELLIPSOID_GM_P233_O086796_CF_F03_0024.hdf"
hdf_file_fp = os.path.join(hdf_dir, hdf_file_label)

os.path.isfile(hdf_file_fp)

False

In [12]:
hdf_file_fp = os.path.join(hdf_dir,hdf_file)
print(os.path.isfile(hdf_file_fp))
print(hdf_file_fp)

NameError: name 'hdf_file' is not defined

In [13]:
input_hdf = MtkFile(hdf_file_fp)

OSError: Trouble opening file: /Volumes/Ehsan-7757225325/2016/april_2016/hdf_files_April2016_from_PH/MISR_AM1_GRP_ELLIPSOID_GM_P233_O086796_CF_F03_0024.hdf

In [14]:
input_hdf.block

NameError: name 'input_hdf' is not defined

In [15]:
input_hdf.grid

NameError: name 'input_hdf' is not defined

In [16]:
input_hdf.grid_list

NameError: name 'input_hdf' is not defined

In [17]:
input_hdf.grid("RedBand").field_list

NameError: name 'input_hdf' is not defined

In [18]:
input_hdf.grid("RedBand").native_field_list

NameError: name 'input_hdf' is not defined

In [19]:
r = MtkRegion(int(path_num), start_block, end_block)
type(r)

MisrToolkit.MtkRegion

In [20]:
# find block corners

corners = path_block_range_to_block_corners(int(path_num), start_block, end_block)

In [21]:
# type(corners.block[1].ctr)
# dir(corners.block[9])

In [22]:
print(corners.block[corners_for_block].ulc)
print(corners.block[corners_for_block].urc)
print(corners.block[corners_for_block].llc)
print(corners.block[corners_for_block].lrc)

(0.000000,0.000000)
(0.000000,0.000000)
(0.000000,0.000000)
(0.000000,0.000000)


In [23]:
# print(bls_to_latlon(int(path_num), 275, 9, 0, 0))
# print(bls_to_latlon(int(path_num), 275, 9, 0, 2047))
# print(bls_to_latlon(int(path_num), 275, 9, 511, 0)) # llc
# print(bls_to_latlon(int(path_num), 275, 9, 511, 2047))

In [24]:
input_hdf.grid("RedBand").field("Red Radiance/RDQI").fill_value

NameError: name 'input_hdf' is not defined

## note: 65515 is fill value for padding part of the image = white part?

In [25]:
redband_img = input_hdf.grid("RedBand").field("Red Radiance/RDQI").read(r).data()

NameError: name 'input_hdf' is not defined

In [26]:
print("redBand min: %d" %redband_img.min())
print("redBand med: %d" %np.median(redband_img))
print("redBand max: %d" %redband_img.max())

NameError: name 'redband_img' is not defined

In [27]:
redband_img.shape

NameError: name 'redband_img' is not defined

In [28]:
print(redband_img.dtype)
print(type(redband_img))

NameError: name 'redband_img' is not defined

In [29]:
redband_img_flat = redband_img.flatten()
redband_img_flat.shape

NameError: name 'redband_img' is not defined

In [30]:
# np.histogram(redband_img)
ret_hist = plt.hist(redband_img_flat, bins='auto')
plt.show()

NameError: name 'redband_img_flat' is not defined

In [31]:
# def rescale(in_arr):
#     domain = in_arr.max()-in_arr.min()
#     for i in 

In [32]:
plot_here_and_write(redband_img, hdf_dir)


NameError: name 'redband_img' is not defined

## plot images using atmmodel.csv 

In [33]:
# # atmmodel_csv = 'atmmodel_july_2016_1to16.csv'
# # home_dir = '/Users/ehsanmos/MLP_dataset/july_2016'

# # april
# atmmodel_csv = 'atmmodel_april_2016.csv'
# home_dir = '/Volumes/Ehsan-7757225325/2016/april_2016/atmmodel'

# # july
# # atmmodel_csv = 'atmmodel_july_2016_1to16.csv'
# # home_dir = '/Users/ehsanmos/MLP_dataset/july_2016'


# in_csv = os.path.join(home_dir, atmmodel_csv)

In [34]:
# # find unique POB from atmmodel.csv 

# list_of_cols = ['#path',' orbit', ' img_block']
# pob_unique = find_unique_pob(in_csv, list_of_cols)
# pob_unique[['#path',' orbit',' img_block']]

In [35]:
# # write out the unique POB info to a file
# output_dir = home_dir
# output_filename = 'uniques_pob_withATMinThem_from_atmmodel.csv'

# pob_unique[['#path',' orbit',' img_block']].to_csv(os.path.join(output_dir, output_filename), index=False)

In [36]:
# # check/visualize unique images in a loop

# for row_indx in range(pob_unique.shape[0]):
    
#     path_num = str(pob_unique['#path'].iloc[row_indx])
#     orbit_num = str(pob_unique[' orbit'].iloc[row_indx])
#     start_block = str(pob_unique[' img_block'].iloc[row_indx])
#     end_block = start_block
#     cam = "AN"
    
#     hdf_file = "MISR_AM1_GRP_ELLIPSOID_GM_P"+path_num+"_O0"+str(orbit_num)+"_"+cam+"_F03_0024.hdf"
#     print('P:%s O:%s B:%s' %(path_num, orbit_num, start_block))
#     hdf_file_fp = os.path.join(hdf_dir, hdf_file)
#     hdf_file_stat = os.path.isfile(hdf_file_fp)
# #     print(hdf_file_stat)
#     if (hdf_file_stat==False):
#         print('hdf file not found!')
#         continue
#     # open (allocate pointer) and read hdf file 
#     input_hdf = MtkFile(hdf_file_fp)
#     input_hdf.block
#     r = MtkRegion(int(path_num), int(start_block), int(end_block))
#     type(r)
#     redband_img = input_hdf.grid("RedBand").field("Red Radiance/RDQI").read(r).data()
#     print('path: %s orbit: %s block: %s' %(path_num, orbit_num, start_block))
#     plot_here_and_write(redband_img, hdf_dir)
#     # need to close the hdf-file here?


# print('successfully printed %s files!' %pob_unique.shape[0])
